In [1]:
import numpy as np
import matplotlib.pyplot as plt
import napari
import pandas as pd
from anndata import AnnData, read_h5ad
import scanpy as sc
import matplotlib as mpl
from scipy.interpolate import interp1d
from tqdm import tqdm
from glob import glob
from pathlib import Path
import pickle
from diptericin_paper_utils import emd_from_line_dist


In [2]:
%matplotlib qt
#%matplotlib inline

In [3]:
"""plot style"""
linewidth = 4
mpl.rc('axes', linewidth=linewidth)
mpl.rc('font', family='Arial')
fontsize = 24
mpl.rc('pdf', fonttype=42)


colors = {'no_inj': [0.8, 0.8, 0.8],
         'mock': [0.4, 0.4, 0.4],
         'e.coli': [0, 0.4, 0],
         'complete': [0, 0.8, 0],
         'st': np.array([70.0, 130.0, 180.0]) / 255}

def style_axes(ax, fontsize=24):
    plt.minorticks_off()
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.xaxis.set_tick_params(labelsize=20)
    ax.yaxis.set_tick_params(labelsize=20)
    for tick in ax.xaxis.get_major_ticks():
        tick.label1.set_fontsize(fontsize)
    for tick in ax.yaxis.get_major_ticks():
        tick.label1.set_fontsize(fontsize)
    plt.tight_layout()
    
    return ax


In [4]:
def bin_aps(aps, bins):
    _counts, bins = np.histogram(aps, bins)
    bins = bins[1:]
    binned_aps = np.zeros_like(aps)
    for i in range(len(binned_aps)):
        binned_aps[i] = get_ap_bin(aps[i], bins)
    
    return binned_aps
    

def get_ap_bin(this_ap, bins):
    this_bin = np.where(np.abs(this_ap - bins) == np.nanmin(np.abs(this_ap - bins)))[0][0]

    return this_bin


def line_dist(gene, adata, key, n_bootstraps=10, layer='md_norm_counts'):
    this_gene_index = np.where(adata.var_names == gene)[0][0]
    ap = np.unique(adata.obs[key])
    X = adata.layers[layer]

    # arrays for storing output
    mean_expression = np.zeros(len(np.unique(ap)))
    std_dev = np.zeros(len(np.unique(ap)))
    bootstrapped_expression = np.zeros((n_bootstraps, len(np.unique(ap))))
    n_cells = X.shape[0]
   
    
    for i in range(len(np.unique(ap))):
        these_cell_indices = adata.obs[key] == ap[i]
        #mean_expression[i] = 10 ** np.mean(X[these_cell_indices, this_gene_index]) - 1
        mean_expression[i] = np.mean(X[these_cell_indices, this_gene_index])
        #mean_expression[i] = np.sum(X[these_cell_indices, this_gene_index]) / np.sum(X[these_cell_indices])
    
    for n in range(n_bootstraps):
        scrambled_ap = np.random.choice(adata.obs[key], size=n_cells, replace=False)
        for i in range(len(np.unique(ap))):
            these_cell_indices = np.random.choice(np.where(adata.obs[key] == ap[i])[0], size=np.sum(adata.obs[key] == ap[i]))
            #bootstrapped_expression[n, i] = np.mean(X[these_cell_indices, this_gene_index])
            bootstrapped_expression[n, i] =  np.mean(X[these_cell_indices, this_gene_index])
            #bootstrapped_expression[n, i] = np.sum(X[these_cell_indices, this_gene_index]) / np.sum(X[these_cell_indices])
    
    std_dev = np.std(bootstrapped_expression, axis=0) 

    return mean_expression, std_dev, ap

                            

    
                


## Load and preprocess the data

In [5]:
"""download the flysta3d data from: https://db.cngb.org/stomics/flysta3d/download/"""
file_path = r'/media/brandon/Data2/Brandon/fly_immune/Flysta3d/L3_b_count_normal_stereoseq.h5ad'
adata = read_h5ad(file_path)

In [6]:
"""create AP bins"""
n_y_bins = 5
all_ys = np.array(adata.obs['new_y'])
y_bins = np.linspace(np.min(all_ys), np.max(all_ys), n_y_bins)
print(f'The anterior-posterior axis goes from {np.min(all_ys)} to {np.max(all_ys)}')

The anterior-posterior axis goes from -178.66514999999998 to 193.73085


In [7]:
"""filter reads to 5% detection"""
detection_percent = np.sum(adata.layers['raw_counts'] > 0, axis=0) / len(adata.layers['raw_counts'])
detection_sel = detection_percent > 0.05
adata = adata[:, detection_sel]

"""get selection of just fat body cells"""
annotation = np.array(adata.obs['annotation'])
fb_sel = (annotation == 'fat body')
#fb_sel = (annotation == 'carcass')
#fb_sel = (annotation == 'muscle')

adata = adata[fb_sel]

#"""replace the log1p-normalized dataset with analytic pearson residuals. store log1p as a new layer"""
#adata.layers['log1p'] = adata.X
"""store the original X as a layer. create another layer that is straight forward log1pPF (log1((c/np.mean(c) + 1)))"""
adata.layers['sct'] = adata.X
mean_depth = np.mean(np.sum(adata.layers['raw_counts'], axis=1))
n_genes = adata.shape[1]
C = adata.layers['raw_counts']
adata.layers['raw_freqs'] = C / np.expand_dims(np.sum(C, axis=1), axis=1)
adata.layers['log1p'] = np.log10(C / np.expand_dims(np.sum(C, axis=1), axis=1) * mean_depth + 1)
adata.layers['md_norm_counts'] = C / np.expand_dims(np.sum(C, axis=1), axis=1) * mean_depth
adata.layers['md_norm_freq_pseudocount'] = (C / np.expand_dims(np.sum(C, axis=1), axis=1) * mean_depth + 1) / (mean_depth + adata.shape[1])
adata.layers['log_md_norm_freq_pseudocount'] = np.log10((C / np.expand_dims(np.sum(C, axis=1), axis=1) * mean_depth + 1) / (mean_depth + adata.shape[1]))
adata.layers['log2_md_norm_freq_pseudocount'] = np.log2((C / np.expand_dims(np.sum(C, axis=1), axis=1) * mean_depth + 1) / (mean_depth + adata.shape[1]))
adata.X = sc.experimental.pp.normalize_pearson_residuals(adata, layer='raw_counts', theta=50, inplace=False)['X']

/tmp/ipykernel_551417/965290870.py:17: ImplicitModificationWarning: Setting element `.layers['sct']` of view, initializing view as actual.
  adata.layers['sct'] = adata.X


In [8]:
"""bin the AP axis"""
ys = np.array(adata.obs['new_y'])
binned_ys = bin_aps(ys, y_bins)
ap_labels = ['0-25%', '25-50%', '50-75%', '75-100%']
adata.obs['anterior_posterior_position'] = [ap_labels[int(b)] for b in binned_ys]
adata.obs['anterior_posterior_position'] = adata.obs['anterior_posterior_position'].astype('category')

"""create finer ap bins for plotting"""
n_y_bins = 16
fine_y_bins = np.linspace(np.min(all_ys), np.max(all_ys), n_y_bins)
binned_ys = bin_aps(ys, fine_y_bins)
fraction_ap = np.linspace(0, 1, n_y_bins - 1)
adata.obs['anterior_posterior_position_fine'] = [fraction_ap[int(b)] for b in binned_ys]

In [13]:
adata

AnnData object with n_obs × n_vars = 6628 × 6505
    obs: 'slice_ID', 'raw_x', 'raw_y', 'new_x', 'new_y', 'new_z', 'annotation', 'anterior_posterior_position', 'anterior_posterior_position_fine'
    uns: 'annotation_colors'
    obsm: 'X_umap', 'spatial'
    layers: 'raw_counts', 'sct', 'raw_freqs', 'log1p', 'md_norm_counts', 'md_norm_freq_pseudocount', 'log_md_norm_freq_pseudocount', 'log2_md_norm_freq_pseudocount'

## Clustering and UMAP

In [9]:
"""run the pre-clustering and clustering routines"""
sc.tl.pca(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)
sc.tl.leiden(adata, n_iterations=2, resolution=0.04, flavor='igraph')

/home/brandon/anaconda3/envs/scanpy/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Plot UMAPs colored by both clusters and AP position

In [66]:
"""colored by cluster"""
sc.pl.umap(adata, color='leiden', legend_fontsize=0.75 * fontsize)
plt.xlabel('')
plt.ylabel('')
plt.title('leiden subclusters', fontsize=fontsize)
ax = style_axes(plt.gca())


In [26]:
plt.savefig(r'/home/brandon/Documents/Code/diptericin-paper/figures/Fig6-SpatialTranscriptomics/L3_fat_body_subclusters_umap.pdf')

## View clusters spatially within the larva

In [ ]:
"""launch a napari viewer with the cells colored by leiden cluster"""
# get colormap to match UMAP clusters. For now, the default mpl color cycle
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

# extract the points of each cell and swap x and z coordinates so to comply with naparis zyx convention.
points = adata.obsm['spatial']
points[:, [0,2]] = points[:, [2,0]]

# view each cluster's points at a time. convert to a DataFrame first for convenience
points_df = pd.DataFrame({'z': points[:,0], 'y': points[:,1], 'x':points[:,2], 'cluster':adata.obs['leiden'].values})
viewer = napari.Viewer()
for i, p in enumerate(points_df.cluster.unique()):
    these_points = points_df[points_df.cluster == p].get(['z', 'y', 'x']).values
    viewer.add_points(these_points, size=2, face_color=colors[i], name=f'subcluster {i}')

# widen the z coordinate for clarity
for layer in viewer.layers:
    layer.scale = (5, 1, 1)

viewer.dims.ndisplay = 3

In [28]:
"""function for viewing a map of a gene signal in 3D space"""
def view_gene_map(gene, adata, viewer=None, layer=None):
    """pass gene=None to create an array with constant intensity at each cell.
       pass gene=all to create an array with intensity = sum of all read counts in each cell."""
    if viewer is None:
        viewer = napari.Viewer()
        
    # extract points of each cell and convert to integer
    points = adata.obsm['spatial'].astype('int')
    
    # shift coordinates to usual image frame (i.e., no negatives)
    points[:, 0] = points[:, 0] + np.abs(np.min(points[: , 0]))
    points[:, 1] = points[:, 1] + np.abs(np.min(points[: , 1]))
    points[:, 2] = points[:, 2] + np.abs(np.min(points[: , 2]))
    
    # create an array that will become the image
    n0, n1, n2 = np.max(points, axis=0)
    grid = np.zeros((n0 + 1, n1 + 1, n2 + 1))
    
    # loop over points and store the coutn value in the array
    for i, point in enumerate(points):
        if gene is None:
            grid[points[i, 0], points[i, 1], points[i, 2]] = 1 
        elif gene == 'all':
            grid[points[i, 0], points[i, 1], points[i, 2]] = np.sum(adata[i].layers['raw_counts'])
        else:
            if layer is None:
                grid[points[i, 0], points[i, 1], points[i, 2]] = 10 ** adata[i, gene].layers['log1p'] - 1
            else:
                grid[points[i, 0], points[i, 1], points[i, 2]] = adata[i, gene].layers[layer]
    
    viewer.add_image(grid, name=gene)
    # widen the z coordinate for clarity
    for layer in viewer.layers:
        layer.scale = (1, 1, 5)

    return viewer
    

In [29]:
"example: "
viewer = view_gene_map(None, adata)
viewer = view_gene_map('TotA', adata, viewer=viewer, layer='md_norm_counts')
viewer.layers[0].contrast_limits = [0, 3]
viewer.layers[1].colormap = 'magma'
viewer.layers[1].blending = 'additive'
viewer.dims.ndisplay = 3
viewer.camera.angles = (139, -42, 70)

/tmp/ipykernel_195252/1967720826.py:30: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  grid[points[i, 0], points[i, 1], points[i, 2]] = adata[i, gene].layers[layer]


## Identify marker genes that separate subclusters
While we used pearson residuals to identify subclusters, we use regular log1p normalized counts for differential expression analysis

In [10]:
"""identify and filter marker genes"""
sc.tl.rank_genes_groups(adata, groupby='leiden', method="wilcoxon", corr_method='bonferroni', layer='sct', pts=True)


In [11]:
sc.tl.filter_rank_genes_groups(adata, groupby='leiden', min_fold_change=1, min_in_group_fraction=0, max_out_group_fraction=1)

In [12]:
sc.pl.rank_genes_groups_heatmap(
    adata, groupby='leiden', key='rank_genes_groups_filtered', n_genes=5, vmin=0, vmax=10, show_gene_labels=True, min_logfoldchange=1)

In [68]:
sc.pl.rank_genes_groups_matrixplot(
    adata, groupby='leiden', key='rank_genes_groups_filtered', n_genes=5, vmin=0, vmax=10,  
    min_logfoldchange=1, cmap='magma')

In [69]:
plt.savefig(r'/home/brandon/Documents/Code/diptericin-paper/figures/Fig6-SpatialTranscriptomics/cluster_matrix_plot.pdf')

## Volcano plot of differentially-expressed genes

In [12]:
imm_genes_table = pd.read_excel(r'/home/brandon/Downloads/Supplementary_List_of_Drosophila_immune_genes.xls', header=6)
imm_genes = imm_genes_table['Symbol'].dropna().tolist()
imm_genes = [g for g in imm_genes if len(g) < 20]

In [13]:
lfc = adata.uns['rank_genes_groups']['logfoldchanges']['2']
pvals = adata.uns['rank_genes_groups']['pvals']['2']
genes2 = adata.uns['rank_genes_groups']['names']['2']
pvals[pvals == 0] = np.min(pvals[pvals > 0] / 10)
pvals_adj = adata.uns['rank_genes_groups']['pvals_adj']['2']


plt.figure()
# not significant
selection = (pvals_adj > 0.05) | (np.abs(lfc) < 0.5)
plt.plot(lfc[~selection], -np.log10(pvals[~selection]), 'mo', markersize=3, alpha=0.5)

# optional -- highlight known immunity genes
genes2_filtered = genes2[~selection]
lfc_filtered = lfc[~selection]
pvals_filtered = pvals[~selection]
pvals_adj_filtered = pvals_adj[~selection]

genes2_imm = [g for g in genes2_filtered if g in imm_genes]
lfc_imm = []
pvals_imm = []
for i, g in enumerate(genes2_filtered):
    if g in imm_genes:
        lfc_imm.append(lfc_filtered[i])
        pvals_imm.append(pvals_filtered[i])

#plt.plot(lfc_imm, -np.log10(pvals_imm), 'go', markersize=4, alpha=0.5)
plt.xlabel('log2 fold change', fontsize=fontsize)
plt.ylabel('-log10 p-value', fontsize=fontsize)
ax = style_axes(plt.gca())

In [ ]:
# save
#plt.savefig(r'/home/brandon/Documents/Code/diptericin-paper/figures/Fig6-SpatialTranscriptomics/volcano_plot.pdf')

In [16]:
"""create table (dataframe) of signficantly differentially expressed genes"""
df = pd.DataFrame({'gene': genes2_filtered, 'log2_fold_change': lfc_filtered, 'p_val': pvals_filtered, 'adj_p_val': pvals_adj_filtered})

# save
#df.to_csv(r'/home/brandon/Documents/Code/diptericin-paper/st_tables/region2_DE_genes_v2.csv')

## Cross reference with immunity genes

In [17]:
print('anterior:')
print(f'num. DE genes = {len(genes2_filtered[lfc_filtered > 0])}')
print(f'num. known immunity genes = {len(imm_genes)}')
print(f'num. shared genes = {len(list(set(imm_genes) & set(genes2_filtered[lfc_filtered > 0])))}')

anterior:
num. DE genes = 721
num. known immunity genes = 564
num. shared genes = 31


In [18]:
# posterior--- group 3
lfc = adata.uns['rank_genes_groups']['logfoldchanges']['3']
pvals = adata.uns['rank_genes_groups']['pvals']['3']
genes3 = adata.uns['rank_genes_groups']['names']['3']
pvals[pvals == 0] = np.min(pvals[pvals > 0] / 10)
pvals_adj = adata.uns['rank_genes_groups']['pvals_adj']['3']


plt.figure()
# not significant
selection = (pvals_adj > 0.05) | (np.abs(lfc) < 0.5)
plt.plot(lfc[~selection], -np.log10(pvals_adj[~selection]), 'mo', markersize=3, alpha=0.5)

# highlight known immunity genes
genes3_filtered = genes3[~selection]
lfc_filtered = lfc[~selection]
pvals_filtered = pvals[~selection]
pvals_adj_filtered = pvals_adj[~selection]


genes3_imm = [g for g in genes3_filtered if g in imm_genes]
lfc_imm = []
pvals_imm = []
pvals_adj_imm = []
for i, g in enumerate(genes3_filtered):
    if g in imm_genes:
        lfc_imm.append(lfc_filtered[i])
        pvals_imm.append(pvals_filtered[i])
        pvals_adj_imm.append(pvals_adj_filtered[i])

plt.plot(lfc_imm, -np.log10(pvals_adj_imm), 'go', markersize=4, alpha=0.5)


plt.xlabel('log2 fold change', fontsize=fontsize)
plt.ylabel('-log10 p-value', fontsize=fontsize)
plt.title('posterior', fontsize=fontsize)
plt.xlim(-4, 4)
ax = style_axes(plt.gca())

/tmp/ipykernel_551417/231102574.py:12: RuntimeWarning: divide by zero encountered in log10
  plt.plot(lfc[~selection], -np.log10(pvals_adj[~selection]), 'mo', markersize=3, alpha=0.5)
/tmp/ipykernel_551417/231102574.py:31: RuntimeWarning: divide by zero encountered in log10
  plt.plot(lfc_imm, -np.log10(pvals_adj_imm), 'go', markersize=4, alpha=0.5)


In [21]:
"""create table (dataframe) of signficantly differentially expressed genes"""
df = pd.DataFrame({'gene': genes3_filtered, 'log2_fold_change': lfc_filtered, 'p_val': pvals_filtered, 'adj_p_val': pvals_adj_filtered})

# save
#df.to_csv(r'/home/brandon/Documents/Code/diptericin-paper/st_tables/region3_DE_genes_v2.csv')

In [17]:
print('posterior:')
print(f'num. DE genes = {len(genes3_filtered[lfc_filtered > 0])}')
print(f'num. known immunity genes = {len(imm_genes)}')
print(f'num. shared genes = {len(list(set(imm_genes) & set(genes3_filtered[lfc_filtered > 0])))}')

posterior:
num. DE genes = 242
num. known immunity genes = 564
num. shared genes = 17


## Plot AP-distribution of upregulated known immune genes

In [14]:
%matplotlib qt
"""anterior-biased"""
plot_genes = ['TotA', 'Nurf-38', 'lwr']
fig, axs = plt.subplots(1, 3, figsize=(10, 3))
counter = 0
color = colors['mock']
scale_factor = 100
for i in range(axs.shape[0]):
    gene = plot_genes[counter]
    plot_ap = np.unique(adata[:, f'{gene}'].obs['anterior_posterior_position_fine'].values)
    mean_expression, std_dev, ap = line_dist(
    gene, adata, key='anterior_posterior_position_fine', n_bootstraps=100, layer='raw_freqs')
    mean_expression = mean_expression
    l = mean_expression - std_dev
    u = mean_expression + std_dev
    ax = axs[i]
    ax.fill_between(plot_ap, l * scale_factor, u * scale_factor, facecolor=color, alpha=0.2)
    ax.plot(plot_ap, mean_expression * scale_factor, '-', color=color, linewidth=4)
    ax.set_title(f'{gene}', fontsize=fontsize)
    ax.set_xlim([0, 1])
    ax = style_axes(ax)
    if counter == 1:
        ax.set_xlabel('fraction of anterior-posterior axis', fontsize=fontsize)
    if counter == 0:
        ax.set_ylabel('mean frequency\n(x$10^{-2}$)', fontsize=fontsize)
    counter += 1
fig.tight_layout()

In [49]:
mean_expression

array([-3.87671685, -3.84311676, -3.83616233, -3.81944466, -3.89144182,
       -3.95543361, -3.96959496, -3.97797155, -3.94884801, -3.92167521,
       -3.9607048 , -3.9680469 , -3.9549942 , -3.95450306, -4.12176704])

In [15]:
plt.savefig(r'/home/brandon/Documents/Code/diptericin-paper/figures/Fig6-SpatialTranscriptomics/anterior_immune_genes_freq.pdf')

In [16]:
%matplotlib qt
"""posterior-biased"""
plot_genes = ['CG13905', 'CG15293', 'CG14957'] 
fig, axs = plt.subplots(1, 3, figsize=(10, 3))
counter = 0
color = colors['mock']
scale_factor = 100
for i in range(axs.shape[0]):
    gene = plot_genes[counter]
    plot_ap = np.unique(adata[:, f'{gene}'].obs['anterior_posterior_position_fine'].values)
    mean_expression, std_dev, ap = line_dist(
    gene, adata, key='anterior_posterior_position_fine', n_bootstraps=100, layer='raw_freqs')
    mean_expression = mean_expression 
    l = mean_expression - std_dev
    u = mean_expression + std_dev
    ax = axs[i]
    ax.fill_between(plot_ap, l * scale_factor, u * scale_factor, facecolor=color, alpha=0.2)
    ax.plot(plot_ap, mean_expression * scale_factor, '-', color=color, linewidth=4)
    ax.set_title(f'{gene}', fontsize=fontsize)
    ax.set_xlim([0, 1])
    #ax.set_ylim([0, 1.2])
    ax = style_axes(ax)
    if counter == 1:
        ax.set_xlabel('fraction of anterior-posterior axis', fontsize=fontsize)
    if counter == 0:
        ax.set_ylabel('mean frequency\n(x$10^{-2}$)', fontsize=fontsize)
    counter += 1
fig.tight_layout()

In [17]:
plt.savefig(r'/home/brandon/Documents/Code/diptericin-paper/figures/Fig6-SpatialTranscriptomics/posterior_immune_genes_freq.pdf')

## Plot house keeping genes along AP

In [18]:
"""plot house keeping genes along AP"""
scale_factor = 1
gene = 'Act5C'
color = np.array([138,43,226]) / 255
plot_ap = np.arange(0, n_y_bins - 1)
plt.figure(figsize=(8,4.5))
mean_expression, std_dev, ap = line_dist(
            gene, adata, key='anterior_posterior_position_fine', n_bootstraps=100, layer='raw_freqs')

m = mean_expression * scale_factor
s = std_dev * scale_factor
l = m - s
u = m + s
plt.fill_between(ap[plot_ap] , l[plot_ap], u[plot_ap], facecolor=color, alpha=0.2, label='_nolabel_')
plt.plot(ap[plot_ap],m[plot_ap], '-', color=color, linewidth=4, label=gene)

gene = 'betaTub56D'
color = 'c'
mean_expression, std_dev, ap = line_dist(
            gene, adata, key='anterior_posterior_position_fine', n_bootstraps=100, layer='raw_freqs')
m = mean_expression * scale_factor
s = std_dev * scale_factor
l = m - s
u = m + s
plt.fill_between(ap[plot_ap] , l[plot_ap], u[plot_ap], facecolor=color, alpha=0.2, label='_nolabel_')
plt.plot(ap[plot_ap],m[plot_ap], '-', color=color, linewidth=4, label=gene)

gene = 'alphaTub84B'
color = 'y'
mean_expression, std_dev, ap = line_dist(
            gene, adata, key='anterior_posterior_position_fine', n_bootstraps=100, layer='raw_freqs')
m = mean_expression * scale_factor
s = std_dev * scale_factor
l = m - s
u = m + s
plt.fill_between(ap[plot_ap] , l[plot_ap], u[plot_ap], facecolor=color, alpha=0.2, label='_nolabel_')
plt.plot(ap[plot_ap],m[plot_ap], '-', color=color, linewidth=4, label=gene)


plt.xlabel('fraction of anterior-posterior axis', fontsize=fontsize)
plt.ylabel('mean frequency', fontsize=fontsize)
plt.legend(fontsize=0.75*fontsize)
plt.ylim([0, 0.005])
ax = style_axes(plt.gca())


In [19]:
plt.savefig(r'/home/brandon/Documents/Code/diptericin-paper/figures/Fig6-SpatialTranscriptomics/house_keeping_genes_freq.pdf')

## Plot known posterior-biased genes along ap

In [22]:
wasp_genes = ['abd-A', 'CG43124', 'CG34436', 'Tep2', 'Stat92E', 'Socs36E']
scale_factor = 10000
plt.figure(figsize=(8.35, 9.12))
for i, gene in enumerate(wasp_genes):
    plt.subplot(3,2,i+1)
    color = mpl.colormaps['Dark2'].colors[i]
    mean_expression, std_dev, ap = line_dist(
            gene, adata, key='anterior_posterior_position_fine', n_bootstraps=100, layer='raw_freqs')
    m = mean_expression * scale_factor
    s = std_dev * scale_factor
    l = m - s
    u = m + s
    plt.fill_between(ap[plot_ap] , l[plot_ap], u[plot_ap], facecolor=color, alpha=0.2, label='_nolabel_')
    plt.plot(ap[plot_ap],m[plot_ap], '-', color=color, linewidth=4, label=gene)
   

    if i == 4:
        plt.xlabel('                       fraction of anterior-posterior axis', fontsize=fontsize)
    if i == 2:
        plt.ylabel('mean frequency (x$10^{-4}$)', fontsize=fontsize)
    plt.title(gene, fontsize=fontsize)
    ax = style_axes(plt.gca())
plt.tight_layout()

In [23]:
plt.savefig(r'/home/brandon/Documents/Code/diptericin-paper/figures/Fig6-SpatialTranscriptomics/wasp_posterior_genes_freq.pdf')